# Combining Sub-aperture and tilted Rowland geometry to multiplex spectral channels

### or - as Dave called it -

# A Doubly-Tilted Rowland Geometry Spectrometer Design

In [ ]:
import os

import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord

In [ ]:
import marxs
from marxs.missions import chandra
from marxs.missions.chandra.hess import HETG
import marxs.visualization.mayavi
from marxs import optics, source

In [ ]:
# Setting up mayavi for notebook output
from mayavi import mlab
# Without setting width and height it all compresses into a sinle pixel for some reason
mlab.init_notebook(width=800, height=600)

In [ ]:
# After I've loaded Mayavi, I can also load matplotlib without screwing things up
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib as mpl

In [ ]:
keeppos = marxs.simulator.KeepCol('pos')
marxm = optics.MarxMirror(os.path.join(os.path.dirname(optics.__file__), 'hrma.par'))
hetg = HETG()
aciss = chandra.ACIS(chips=[4,5,6,7,8,9], aimpoint=chandra.AIMPOINTS['ACIS-S'])
chand = chandra.Chandra(elements=[marxm, hetg, aciss], postprocess_steps = [keeppos])

In [ ]:
en = np.arange(0.5, 7., .5)
flux = en**(-2)

# Input as dictionary
dictspectrum = {'energy': en, 'flux': flux}

star = source.PointSource((30., 30.), energy=2., flux= 1.)
pointing = source.FixedPointing(coords=(30., 30.))

photons = star.generate_photons(20000)
photons = pointing(photons)
photons = chand(photons)
ind = (photons['probability'] > 0) & (photons['facet'] >=0) & np.isfinite(photons['tdetx'])


In [ ]:
# Color all hetg gratings according to the sector they belong to
sectorcol = dict(zip('ABCDEFG', plt.cm.gist_rainbow(np.linspace(0, 1, 6))[:, :3]))
for e in hetg.elements:
    e.display = {'color': sectorcol[e.name[1]]}
# Color all photons according to the sector they go through
photons['color'] = [sectorcol[hetg.elements[int(i)].name[1]] for i in photons['facet']]

In [ ]:
posdat = marxs.visualization.utils.format_saved_positions(keeppos)[ind, :, :]

fig = mlab.figure()
chand.plot(format='mayavi', viewer=fig)
marxs.visualization.mayavi.plot_rays(posdat, scalar=photons['energy'][ind])
fig

In [ ]:
pp = photons[ind]
fig = plt.figure()
ax0 = fig.add_subplot(121, axisbg='grey')
ax1 = fig.add_subplot(122, axisbg='gray')
for p in pp [(pp['tdetx'] > 4000)&(pp['tdetx'] < 4150)]:
    ax0.plot(p['tdetx']-4136, p['tdety']-2232, '.', c=p['color'])
ax0.set_aspect("equal")
ax0.set_xlim([0, 3])
ax0.set_ylim([0, 4])
ax0.set_title('0 th order')

for p in pp [(pp['tdetx'] > 3500)&(pp['tdetx'] < 3800)]:
    ax1.plot(p['tdetx']-3583, p['tdety']-2183, '.', c=p['color'])
ax1.set_aspect("equal")
ax1.set_xlim([0, 3])
ax1.set_ylim([0, 4])
ax1.set_title('1 st order MEG')

In [ ]:
# Make a model that is not to scale for display purposes.
# Pick a small Rowland radius and make diffraction angle much larger.

from marxs.design import rowland
demo_rowland = rowland.RowlandTorus(500, 500)

demo_gas = rowland.GratingArrayStructure(demo_rowland, 25., x_range=[800, 1000], 
                                         radius=[20, 100],
                                         elem_class=marxs.optics.FlatGrating,
                                         elem_args={'d': 1e-6, 'zoom': [1, 10, 10],
                                                   'order_selector': marxs.optics.constant_order_factory(1)}
                                        )
demo_det = rowland.RowlandCircleArray(demo_rowland,
                                      d_element=10.5,
                                      theta=[np.pi - 0.5, np.pi + 0.5],
                                     elem_class= marxs.optics.FlatDetector,
                                     elem_args={'zoom': [1, 5, 5], 'pixsize': 0.01})